In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# 訓練データの読み込み
df_train = pd.read_csv('C:/Users/thisi/OneDrive/デスクトップ/Dateset1.csv')

# 説明変数と目的変数を分離
X_train = df_train.drop('CRIM', axis=1)
y_train = df_train['CRIM']

# パイプラインの定義（正規化とSVR）
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# ハイパーパラメータの範囲を定義
param_grid = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__epsilon': [0.01, 0.1, 0.5],
    'svr__kernel': ['rbf', 'linear'],
    'svr__gamma': ['scale', 'auto', 0.1, 1]
}

# GridSearchCVの設定と実行
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 最適なモデルを取得
best_model = grid_search.best_estimator_

print("最適なパラメータ:", grid_search.best_params_)

# 新しいデータの読み込み（予測したいデータ）
df_new = pd.read_csv('C:/Users/thisi/OneDrive/デスクトップ/Dateset2.csv')

# 新しいデータの説明変数を抽出
X_new = df_new[X_train.columns]  # 訓練データと同じ列を使用

# 予測の実行
y_pred = best_model.predict(X_new)

# 予測結果をデータフレームに追加
df_new['Predicted_Target'] = y_pred

# 予測結果の表示
print("\n予測結果の最初の5行:")
print(df_new[['Predicted_Target'] + list(X_train.columns)].head())

# 予測結果をCSVファイルに保存
csv_filename = 'prediction_results.csv'
df_new.to_csv(csv_filename, index=False)
print(f"\n予測結果を'{csv_filename}'に保存しました。")

最適なパラメータ: {'svr__C': 100, 'svr__epsilon': 0.5, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}

予測結果の最初の5行:
   Predicted_Target    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0         -0.150665  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1          0.471890   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2          0.039490   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3          0.260986   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4          0.171058   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  

予測結果を'prediction_results.csv'に保存しました。
